In [1]:
import pandas as pd
import numpy as np
from cointegracao import Cointegration
from itertools import combinations, permutations
import time
from backtest import Executer
from tqdm.contrib.concurrent import process_map, thread_map # or thread_map
import gc
import pickle

In [2]:
import execute

In [5]:
n_workers = 7 # Número de CPUS que vai usar, na minha máquina ficou bom com 8, começa testando com 1 e ve se vai melhorando
periodo = '5minutos' # Período de teste pode ser 'diario' ou 'minutos' ou '1minutos' ou '5 minutos'

# Período de tempo que vai olhar para tras para calcular a cointegração: 
# no diário uso 252, no de 15 minutos combinamos 3 meses = 15(minutos) * 21(dias uteis) * 3(meses) = 1300, 19500 minutos 3900 de 5minutos
train_size = 3900
significancia = 0.05 # Significancia usada na cointegração
z_score_saida = 0.5 # Z-score que considera retorno a média, quando ele vai sair.

# Z-score de stop quando o resíduo está indo para longe do retorno a média:
# pode ser 'desligado' ou valores acima de 1
z_score_stop = 'desligado'
conf_var = 0.05 # Confiança do VAR. Também aceita 'desligado' para o caso sem stop

usar_halflife = 'desligado' # Usa o stop no halflife. aceita 'ligado' e 'desligado'
usar_var = 'ligado' # Usa o var como stop. Aceita 'ligado' e 'desligado'

In [6]:
if __name__ == '__main__':
    resultados_raw = execute.executa(
        n_workers = n_workers,
        periodo = periodo,
        train_size = train_size,
        significancia = significancia,
        z_score_saida = z_score_saida,
        z_score_stop = z_score_stop,
        conf_var = conf_var,
        usar_halflife = usar_halflife,
        usar_var = usar_var    
    )

Executando 2020-01-01/2020-07-01


  0%|          | 0/5 [00:00<?, ?it/s]

In [9]:
# Calcula o retorno a partir dos resultados gerados no backtest
df_returns = execute.calcula_retornos(resultados_raw)

# Salva um arquivo que preciso para fazer os gráficos e tabelas depois 
filename = f'{periodo}_{train_size}_{significancia}_{z_score_saida}_{z_score_stop}_{conf_var}_{usar_halflife}_{usar_var}.csv'
df_returns.to_csv(filename, index=False)

with open(filename+'.pickle', 'wb') as handle:
    pickle.dump(df_returns, handle)

In [9]:
# Transforma do jeito que vc gosta pra salvar em excel
df_returns.pivot(index='pair', columns='date', values='return').T

pair,"(ABEV3, BRFS3)"
date,
2020-04-16,NaN
2020-04-17,0.010238
2020-04-20,0.007793
2020-04-22,-0.026359
2020-04-23,-0.009454
2020-04-24,0.012082
2020-04-27,-0.070931


In [7]:
# Salva como arquivo de excel. Lembra sempre de escolher um 'nome_arquivo.xlsx' diferente,
# senão ele vai salvar em cima do arquivo existente.

df_returns.pivot(index='pair', columns='date', values='return').T.to_excel(filename+'.xlsx')